In [1]:
import xarray as xr
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import metpy.calc as mpcalc
from metpy.units import units

import pickle
import os

### set some parameters here - the start date/time of your forecast, where your zarr files are, and where you want the images to go.

In [2]:
# ------------------------------------------
init_time = "2020-01-01T18"
init = pd.to_datetime(init_time, format="%Y-%m-%dT%H")

zarrdir = "output_data/"
imagedir = "images"
# ------------------------------------------

init

Timestamp('2020-01-01 18:00:00')

### nice surface temperature colormap (optional); colormap for absolute vorticity

In [3]:
### temperature from pickle
fp = open('sfc_temp_cmap.pkl', 'rb')
sfc_temp_cmap = pickle.load(fp)
fp.close()

### create 500avo colormap
colors1 = plt.cm.YlOrRd(np.linspace(0, 1, 36))
colors2 = plt.cm.BuPu(np.linspace(0.5, 0.75, 8))
colors_500avo = np.vstack((colors2, (1, 1, 1, 1), colors1))

### read in zarrs

In [4]:
ds_sfc = xr.open_zarr(zarrdir+"/"+init.strftime("%Y%m%d%H")+"/pangu_sfc.zarr")
ds_upper = xr.open_zarr(zarrdir+"/"+init.strftime("%Y%m%d%H")+"/pangu_upper.zarr")


## make some plots!

#### map setup

In [5]:
datacrs = ccrs.PlateCarree()

plotcrs = ccrs.LambertConformal(central_longitude=-100, central_latitude=37.5)

def plot_background(ax):
     ### set up bounding box surrounding specified station
    latmin=27.75
    latmax=48.25
    lonmin=-118.7
    lonmax=-83.75

    ax.set_extent([lonmin,lonmax,latmin,latmax])
    ax.coastlines('50m', edgecolor='black', linewidth=0.9)
    ax.add_feature(cfeature.STATES.with_scale('10m'), linewidth=1.2)

    return lonmin,lonmax,latmin,latmax

### 2-m temperature, MSLP, 10-m winds

In [6]:
#for tt in range(0,2):
for tt in range(0,len(ds_sfc.valid_time)):

    fig = plt.figure(figsize=(16,8.6))
    ax = fig.add_subplot(1,1,1,projection=plotcrs)

    lonmin, lonmax, latmin, latmax = plot_background(ax)

    vtime = pd.to_datetime(ds_sfc.valid_time[tt].values)
    mslp = ds_sfc.mslp.sel(valid_time=vtime, lon=slice(lonmin+360-10,lonmax+370), lat=slice(latmax+5,latmin-5))
    u10 = ds_sfc.u10.sel(valid_time=vtime, lon=slice(lonmin+360-10,lonmax+370), lat=slice(latmax+5,latmin-5))
    v10 = ds_sfc.v10.sel(valid_time=vtime, lon=slice(lonmin+360-10,lonmax+370), lat=slice(latmax+5,latmin-5))
    t2m = ds_sfc.t2m.sel(valid_time=vtime, lon=slice(lonmin+360-10,lonmax+370), lat=slice(latmax+5,latmin-5))

    fcst_lead = int((vtime - init) / pd.Timedelta('1 hour'))

    print("lead "+str(fcst_lead)+", valid "+str(vtime))

    clevs = np.arange(-40,110,2)  ### range of temperatures

    lon2d, lat2d = np.meshgrid(mslp.lon,mslp.lat)

    ### MSLP
    cs1 = ax.contour(lon2d, lat2d,
                     mslp/100.,
                            np.arange(900,1100,4), colors='black',
                                 linewidths=2, transform=ccrs.PlateCarree(),
                     transform_first=True)
    plt.clabel(cs1, fontsize=11, inline=True, inline_spacing=5, fmt='%i',
                        rightside_up=True, use_clabeltext=True)

    ### 2-m temp
    cf = ax.contourf(lon2d, lat2d, 1.8*(t2m-273.15)+32.,
                         clevs,
                         cmap=sfc_temp_cmap, extend='both',
                         transform=ccrs.PlateCarree(), transform_first=True)
    cb = plt.colorbar(cf, orientation='horizontal', pad=0.01, aspect=50, shrink=0.6)
    cb.set_label('Temperature (F)', fontsize=10)
    cb.ax.tick_params(labelsize=10)
    cb.ax.locator_params(nbins=20)

    ### winds
    # Transform Vectors before plotting, then plot wind barbs.
    wind_slice = (slice(None, None, 4), slice(None, None, 4))
    ax.barbs(u10[wind_slice].lon, u10[wind_slice].lat,
             u10[wind_slice].values*1.943844, v10[wind_slice].values*1.943844,   ### to knots
             lw=0.9, length=5.5,
             transform=ccrs.PlateCarree())

    ax.set_title("Pangu-weather\nMSLP, 2-m temperature, 10-m winds",
                  loc="left", horizontalalignment='left', fontsize=10.5, fontweight='bold')
    ax.set_title("initialized "+init.strftime("%H%M UTC %a %d %b %Y")+"\n"+str(fcst_lead)+"-h forecast valid "+vtime.strftime("%H%M UTC %a %d %b %Y"),
                  loc="right", horizontalalignment='right', fontsize=9.5)

    outdir = imagedir+"/"+init.strftime("%Y%m%d%H")
    os.system("mkdir -p "+outdir)

    outfile = "t2m_pangu_"+init.strftime("%Y%m%d%H")+"_f"+str(fcst_lead).zfill(3)+".png"

    plt.savefig(outdir+"/"+outfile,
                bbox_inches='tight', transparent=False, facecolor='white', dpi=225)

    #plt.show()

    plt.close('all')

print("done!")

lead 0, valid 2020-01-01 18:00:00
lead 6, valid 2020-01-02 00:00:00
lead 12, valid 2020-01-02 06:00:00
done!


### 850-hPa temperature, heights, winds

In [7]:
for tt in range(0,len(ds_sfc.valid_time)):

    lev = 850.  ### in hPa

    fig = plt.figure(figsize=(16,8.6))
    ax = fig.add_subplot(1,1,1,projection=plotcrs)

    lonmin, lonmax, latmin, latmax = plot_background(ax)

    vtime = pd.to_datetime(ds_upper.valid_time[tt].values)
    z = ds_upper.z.sel(valid_time=vtime, lon=slice(lonmin+360-10,lonmax+370), lat=slice(latmax+5,latmin-5), lev=lev)
    u = ds_upper.u.sel(valid_time=vtime, lon=slice(lonmin+360-10,lonmax+370), lat=slice(latmax+5,latmin-5), lev=lev)
    v = ds_upper.v.sel(valid_time=vtime, lon=slice(lonmin+360-10,lonmax+370), lat=slice(latmax+5,latmin-5), lev=lev)
    t = ds_upper.t.sel(valid_time=vtime, lon=slice(lonmin+360-10,lonmax+370), lat=slice(latmax+5,latmin-5), lev=lev)

    fcst_lead = int((vtime - init) / pd.Timedelta('1 hour'))

    print("lead "+str(fcst_lead)+", valid "+str(vtime))

    fig = plt.figure(figsize=(16,8.6))
    ax = fig.add_subplot(1,1,1,projection=plotcrs)

    ax.set_extent([lonmin,lonmax,latmin,latmax])
    ax.coastlines('50m', edgecolor='black', linewidth=0.9)
    ax.add_feature(cfeature.STATES.with_scale('10m'), linewidth=1.2)

    clevs = np.arange(-40,43,1)  ### range of temperatures

    lon2d, lat2d = np.meshgrid(z.lon,z.lat)

    ### height
    cs1 = ax.contour(lon2d, lat2d,
                     z/9.80665,
                        np.arange(0, 3001, 30), colors='black',
                                 linewidths=2, transform=ccrs.PlateCarree(),
                     transform_first=True)
    plt.clabel(cs1, fontsize=11, inline=True, inline_spacing=5, fmt='%i',
                        rightside_up=True, use_clabeltext=True)

    ### temp
    cf = ax.contourf(lon2d, lat2d, t-273.15,
                         clevs,
                         cmap=sfc_temp_cmap, extend='both',
                         transform=ccrs.PlateCarree(), transform_first=True)
    cb = plt.colorbar(cf, orientation='horizontal', pad=0.01, aspect=50, shrink=0.6)
    cb.set_label('Temperature (C)', fontsize=10)
    cb.ax.tick_params(labelsize=10)

    ### winds
    # Transform Vectors before plotting, then plot wind barbs.
    wind_slice = (slice(None, None, 4), slice(None, None, 4))
    ax.barbs(u[wind_slice].lon, u[wind_slice].lat,
             u[wind_slice].values*1.943844, v[wind_slice].values*1.943844,   ### to knots
             lw=0.9, length=5.5,
             transform=ccrs.PlateCarree())

    ax.set_title("Pangu-weather\n850-hPa temperature, heights, and winds",
                  loc="left", horizontalalignment='left', fontsize=10.5, fontweight='bold')
    ax.set_title("initialized "+init.strftime("%H%M UTC %a %d %b %Y")+"\n"+str(fcst_lead)+"-h forecast valid "+vtime.strftime("%H%M UTC %a %d %b %Y"),
                  loc="right", horizontalalignment='right', fontsize=9.5)

    outdir = imagedir+"/"+init.strftime("%Y%m%d%H")
    os.system("mkdir -p "+outdir)

    outfile = "850temp_pangu_"+init.strftime("%Y%m%d%H")+"_f"+str(fcst_lead).zfill(3)+".png"

    plt.savefig(outdir+"/"+outfile,
                bbox_inches='tight', transparent=False, facecolor='white', dpi=225)

    #plt.show()

    plt.close('all')

lead 0, valid 2020-01-01 18:00:00
lead 6, valid 2020-01-02 00:00:00
lead 12, valid 2020-01-02 06:00:00


### 500-hPa heights, vorticity, winds

In [8]:
for tt in range(0,len(ds_sfc.valid_time)):

    lev = 500.  ### in hPa

    fig = plt.figure(figsize=(16,8.6))
    ax = fig.add_subplot(1,1,1,projection=plotcrs)

    lonmin, lonmax, latmin, latmax = plot_background(ax)

    vtime = pd.to_datetime(ds_upper.valid_time[tt].values)
    z = ds_upper.z.sel(valid_time=vtime, lon=slice(lonmin+360-10,lonmax+370), lat=slice(latmax+5,latmin-5), lev=lev)
    u = ds_upper.u.sel(valid_time=vtime, lon=slice(lonmin+360-10,lonmax+370), lat=slice(latmax+5,latmin-5), lev=lev)
    v = ds_upper.v.sel(valid_time=vtime, lon=slice(lonmin+360-10,lonmax+370), lat=slice(latmax+5,latmin-5), lev=lev)

    fcst_lead = int((vtime - init) / pd.Timedelta('1 hour'))

    print("lead "+str(fcst_lead)+", valid "+str(vtime))

    fig = plt.figure(figsize=(16,8.6))
    ax = fig.add_subplot(1,1,1,projection=plotcrs)

    ax.set_extent([lonmin,lonmax,latmin,latmax])
    ax.coastlines('50m', edgecolor='black', linewidth=0.9)
    ax.add_feature(cfeature.STATES.with_scale('10m'), linewidth=1.2)

    clevs = list(range(-8, 1, 1))+list(range(12, 36, 1))
    lon2d, lat2d = np.meshgrid(z.lon,z.lat)

    ### height
    cs1 = ax.contour(lon2d, lat2d,
                     z/9.80665,
                        np.arange(4800, 6060, 60), colors='black',
                         linewidths=2, transform=ccrs.PlateCarree(),
                         transform_first=True)
    plt.clabel(cs1, fontsize=11, inline=True, inline_spacing=5, fmt='%i',
                        rightside_up=True, use_clabeltext=True)

    ### temp
    cf = ax.contourf(lon2d, lat2d,
                     1e5*mpcalc.absolute_vorticity(u, v),
                         clevs,
                         colors=colors_500avo, extend='both',
                         transform=ccrs.PlateCarree(), transform_first=True)
    cb = plt.colorbar(cf, orientation='horizontal', pad=0.01, aspect=50, shrink=0.6)
    cb.set_label('absolute vorticity ($*10^5$ s$^{-1}$)', fontsize=10)
    cb.ax.tick_params(labelsize=10)

    ### winds
    # Transform Vectors before plotting, then plot wind barbs.
    wind_slice = (slice(None, None, 4), slice(None, None, 4))
    ax.barbs(u[wind_slice].lon, u[wind_slice].lat,
             u[wind_slice].values*1.943844, v[wind_slice].values*1.943844,   ### to knots
             lw=0.9, length=5.5,
             transform=ccrs.PlateCarree())

    ax.set_title("Pangu-weather\n500-hPa absolute vorticity, heights, and winds",
                  loc="left", horizontalalignment='left', fontsize=10.5, fontweight='bold')
    ax.set_title("initialized "+init.strftime("%H%M UTC %a %d %b %Y")+"\n"+str(fcst_lead)+"-h forecast valid "+vtime.strftime("%H%M UTC %a %d %b %Y"),
                  loc="right", horizontalalignment='right', fontsize=9.5)

    outdir = imagedir+"/"+init.strftime("%Y%m%d%H")
    os.system("mkdir -p "+outdir)

    outfile = "500avo_pangu_"+init.strftime("%Y%m%d%H")+"_f"+str(fcst_lead).zfill(3)+".png"

    plt.savefig(outdir+"/"+outfile,
                bbox_inches='tight', transparent=False, facecolor='white', dpi=225)

    #plt.show()

    plt.close('all')

lead 0, valid 2020-01-01 18:00:00
lead 6, valid 2020-01-02 00:00:00
lead 12, valid 2020-01-02 06:00:00
